In [15]:
import glob
import librosa
import os
import sys
from pathlib import Path
from glob import glob
from os.path import join, exists, basename, splitext

## Encoder model数据处理

ST-CMD数据处理

In [1]:
cd /search/hadoop07/wuzijun/voice_data/STcmds

/search/hadoop07/wuzijun/voice_data/STcmds


In [7]:
file_list = glob.glob('./ST-CMDS-20170001_1-OS/*.wav')

In [15]:
splitext(basename(file_list[0]))

('20170001P00099I0019', '.wav')

In [25]:
speaker_list = set([splitext(basename(f))[0][9:15] for f in file_list])

In [26]:
len(speaker_list)

855

In [38]:
for speaker in speaker_list:
    Path(f'./wav/{speaker}').mkdir(exist_ok=True)
    os.system(f'mv ./ST-CMDS-20170001_1-OS/*{speaker}* ./wav/{speaker}/')

Prime数据处理

In [55]:
cd ../Primewords/

/search/hadoop07/wuzijun/voice_data/Primewords


In [40]:
import pandas as pd
import json

In [44]:
js = json.load(open('/search/hadoop07/wuzijun/voice_data/Primewords/set1_transcript.json', 'r'))

In [47]:
wave_data = pd.DataFrame(js)

In [64]:
wave_data.head(10)

,id,file,user_id,text,length
0,64869,d6b26896-f2ef-4085-950d-47cd24f21d8f.wav,1013,于是 一切又重新开始 而这一来 我又忘记了它们,8.64
1,64870,8e1dd32b-a47b-4ed6-bc92-b12050c2284a.wav,0480,黎正训 黎安理 黎恂 黎恺 黎庶焘等都在那里任过教,8.16
2,64871,8787ea1c-4125-4817-9453-2e8e9c616739.wav,0887,韵音令辞是语言优美动听 往辄破的 是说理论上一发即中,12.56
3,64872,2cd32895-41be-4365-a85e-395d6f8c0bf6.wav,0999,富有弹性 也是工业中使用的上等材料,4.8
4,64875,e2b38e1b-fc8b-4602-b293-dbe46b918cf0.wav,0590,我们对这个得谦卑 对那个得谦卑 在这里脱帽,6.72
5,64878,fe24e279-528f-451e-8d80-7e69582d2f73.wav,0480,江西白鹭洲位于江西吉安市区东面的中 长一点五零点五公里,8.16
6,64879,ce294649-575c-4911-a673-c6d33b76301d.wav,0419,白川澄子 本名白土澄子 一九三五年六月二十六日出生于日本,6.72
7,64881,6a292bf1-a4f8-4e71-b964-fab77598b76f.wav,1016,及平江陵之后 王褒入关 贵游等翕然并学褒书,10.44
8,64882,bf7dd9d9-aae2-410a-902c-f56cb6228798.wav,0999,满庭芳带过青江引 叨叨令带过折桂令 山坡羊带过青哥儿,11.28
9,64883,65142109-32a2-43b7-a4dd-34b238b30fdd.wav,0590,遇有军情 守城部队即可通过跑马道迅速到达战斗地点,7.8


In [52]:
max(wave_data.user_id.apply(lambda x: len(str(x))))

4

In [59]:
wave_data.dtypes

id         object
file       object
user_id    object
text       object
length     object
dtype: object

In [60]:
wave_data.user_id = wave_data.user_id.apply(lambda x: str(10000 + int(x))[-4:])

In [65]:
len(set(wave_data.user_id))

296

In [66]:
wave_data.to_csv('./transcript/transcript.txt', header=None, index=None)

In [ ]:
find ./audio_files/ -type f -exec mv {}  ./wav/ \;

In [68]:
for speaker in set(wave_data.user_id):
    Path(f'./wav/{speaker}').mkdir(exist_ok=True)

In [69]:
wave_data[['file', 'user_id']] \
    .apply(lambda x: os.system(f'mv ./wav/{x[0]} ./wav/{x[1]}'), axis=1)

0        0
1        0
2        0
3        0
4        0
        ..
50897    0
50898    0
50899    0
50900    0
50901    0
Length: 50902, dtype: int64

## Synthesizer model数据处理

In [41]:
from pypinyin import pinyin, lazy_pinyin, Style

In [44]:
pinyin('算了吧', style=Style.TONE3)

[['suan4'], ['le'], ['ba']]

In [46]:
from pypinyin.contrib.neutral_tone import NeutralToneWith5Mixin
from pypinyin.converter import DefaultConverter
from pypinyin.core import Pinyin

In [48]:
class MyConverter(NeutralToneWith5Mixin, DefaultConverter):
    pass

In [49]:
my_pinyin = Pinyin(MyConverter())
pinyin = my_pinyin.pinyin
lazy_pinyin = my_pinyin.lazy_pinyin

In [65]:
lazy_pinyin('我不喜欢吃薄荷', style=Style.TONE3)

['wo3', 'bu4', 'xi3', 'huan1', 'chi1', 'bo4', 'he5']

### AIshell

In [66]:
!head -3 /search/hadoop07/wuzijun/voice_data/Biaobei/audio_text_val_filelist.txt

/search/hadoop07/wuzijun/voice_data/Biaobei/Wave/002937.wav|shi4 zhu3 ren2 zai4 cao1 zong4 wo3 de5 xin1 yo5
/search/hadoop07/wuzijun/voice_data/Biaobei/Wave/008617.wav|gou4 fang2 he2 tong4 bai3 zai4 zhuo1 shang4 mai3 jia1 hu2 bing1 chi2 yi2 pian4 ke4 yao3 le5 yao2 zui3 chun2 qian1 ming2 cheng2 jiao1
/search/hadoop07/wuzijun/voice_data/Biaobei/Wave/007130.wav|di4 shi2 jie4 shi4 ji4 lv4 jian3 cha2 wei3 yuan2 hui4 quan2 ti2 wei3 yuan2 lie4 xi2 le5 hui4 yi4


In [13]:
path_aishell = '/search/hadoop07/wuzijun/voice_data/AIshell'

In [70]:
ll $path_aishell/wav/S0002

总用量 55780
-rw-r--r-- 1 suanfa 192012 6月  13 2017 BAC009S0002W0122.wav
-rw-r--r-- 1 suanfa 123724 6月  13 2017 BAC009S0002W0123.wav
-rw-r--r-- 1 suanfa 173132 6月  13 2017 BAC009S0002W0124.wav
-rw-r--r-- 1 suanfa 101388 6月  13 2017 BAC009S0002W0125.wav
-rw-r--r-- 1 suanfa  82476 6月  13 2017 BAC009S0002W0126.wav
-rw-r--r-- 1 suanfa 188524 6月  13 2017 BAC009S0002W0127.wav
-rw-r--r-- 1 suanfa 108620 6月  13 2017 BAC009S0002W0128.wav
-rw-r--r-- 1 suanfa 196396 6月  13 2017 BAC009S0002W0129.wav
-rw-r--r-- 1 suanfa 132204 6月  13 2017 BAC009S0002W0130.wav
-rw-r--r-- 1 suanfa 141036 6月  13 2017 BAC009S0002W0131.wav
-rw-r--r-- 1 suanfa 216556 6月  13 2017 BAC009S0002W0132.wav
-rw-r--r-- 1 suanfa 298380 6月  13 2017 BAC009S0002W0133.wav
-rw-r--r-- 1 suanfa 233228 6月  13 2017 BAC009S0002W0134.wav
-rw-r--r-- 1 suanfa 228844 6月  13 2017 BAC009S0002W0135.wav
-rw-r--r-- 1 suanfa 202380 6月  13 2017 BAC009S0002W0136.wav
-rw-r--r-- 1 suanfa 231980 6月  13 2017 BAC009S0002W0137.wav
-rw-r--r-- 1 suanfa 206478 6月 

In [31]:
!head -3 $path_aishell/transcript/aishell_transcript_v0.8.txt

BAC009S0002W0122 而 对 楼市 成交 抑制 作用 最 大 的 限 购
BAC009S0002W0123 也 成为 地方 政府 的 眼中 钉
BAC009S0002W0124 自 六月 底 呼和浩特 市 率先 宣布 取消 限 购 后


In [24]:
speaker_aishell = glob(f'{path_aishell}wav/*')

In [39]:
len(speaker_aishell)

400

In [32]:
transcripts_aishell = open(f'{path_aishell}/transcript/aishell_transcript_v0.8.txt', 'r').readlines()

In [101]:
def get_pinyin(sentence):
    sentence = sentence.strip().replace(' ', '').replace('\n', '')
    return ' '.join(lazy_pinyin(sentence, style=Style.TONE3))

In [102]:
def get_path(root_path, first):
    speaker_path = first[6:11]
    file_name = first + '.wav'
    return join(root_path, speaker_path, file_name)

In [103]:
aishell_tuples = [(get_path(join(path_aishell, 'wav'), l[:16]), get_pinyin(l[16:]), l[6:11]) for l in transcripts_aishell]

In [104]:
len(aishell_tuples)

141600

In [11]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

In [119]:
aishell_train, aishell_eval = train_test_split(aishell_tuples, test_size=300)

In [121]:
len(aishell_train), len(aishell_eval)

(141300, 300)

In [16]:
aishell_train_df = pd.read_csv(f'{path_aishell}/audio_text_train_filelist.txt', 
                               sep='|', header=None, 
                               names=['wav_path', 'pinyin', 'speaker_name', 'speaker_emb_path'])

In [19]:
aishell_train_df.speaker_emb_path = aishell_train_df.speaker_emb_path.apply(lambda x: x.replace('AIshell/', 'AIshell/embed/'))

In [22]:
aishell_val_df = pd.read_csv(f'{path_aishell}/audio_text_val_filelist.txt', 
                               sep='|', header=None, 
                               names=['wav_path', 'pinyin', 'speaker_name', 'speaker_emb_path'])

In [23]:
aishell_val_df.speaker_emb_path = aishell_val_df.speaker_emb_path.apply(lambda x: x.replace('AIshell/', 'AIshell/embed/'))

In [173]:
aishell_train_df = pd.DataFrame(aishell_train, columns=['wav_path', 'pinyin', 'SpeakerID'])

In [175]:
aishell_eval_df = pd.DataFrame(aishell_eval, columns=['wav_path', 'pinyin', 'SpeakerID'])

In [177]:
aishell_eval_df.head(1)

,wav_path,pinyin,SpeakerID
0,/search/hadoop07/wuzijun/voice_data/AIshell/wa...,ta1 me5n zhen1 de5 neng2 zou3 dao4 yi4 qi3 ma5,S0363


In [209]:
aishell_train_df['speaker_emb_path'] = aishell_train_df[['wav_path', 'SpeakerID']] \
    .apply(lambda x: x[0].replace('wav', 'embed').replace('/' + x[1], ''), axis=1)

In [213]:
aishell_eval_df['speaker_emb_path'] = aishell_eval_df[['wav_path', 'SpeakerID']] \
    .apply(lambda x: x[0].replace('wav', 'embed').replace('/' + x[1], ''), axis=1)

In [25]:
aishell_train_df.to_csv(f'{path_aishell}/audio_text_train_filelist.txt', sep='|', header=False, index=False)

In [27]:
aishell_val_df.to_csv(f'{path_aishell}/audio_text_val_filelist.txt', sep='|', header=False, index=False)

### Magicdata

In [132]:
import pandas as pd

In [29]:
path_magic = '/search/hadoop07/wuzijun/voice_data/Magicdata'

In [139]:
cat $path_magic/transcript/*.txt >> $path_magic/transcript/all_trans.txt

In [141]:
magic_list = pd.read_csv(path_magic + '/transcript/all_trans.txt', sep='\t')

In [143]:
magic_list = magic_list[magic_list.UtteranceID.apply(lambda x: '.wav' in x)]

In [148]:
magic_list.drop_duplicates(subset='UtteranceID', inplace=True)

/search/hadoop02/suanfa/anaconda3/envs/tacotron2/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [149]:
len(magic_list)

609552

In [151]:
magic_list.head(3)

,UtteranceID,SpeakerID,Transcription
0,37_5622_20170913203118.wav,37_5622,高德地图
1,37_5622_20170913203322.wav,37_5622,我想用百度搜索短信
2,37_5622_20170913203824.wav,37_5622,水费还可以用多长时间


In [152]:
path_magic

'/search/hadoop07/wuzijun/voice_data/Magicdata'

In [158]:
magic_list['wav_path'] = magic_list[['UtteranceID', 'SpeakerID']].apply(lambda x: f'{path_magic}/wav/{x[1]}/{x[0]}', axis=1)

/search/hadoop02/suanfa/anaconda3/envs/tacotron2/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [160]:
magic_list['pinyin'] = magic_list.Transcription.apply(lambda x: get_pinyin(x))

/search/hadoop02/suanfa/anaconda3/envs/tacotron2/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [222]:
magic_list['speaker_emb_path'] = magic_list[['wav_path', 'SpeakerID']] \
    .apply(lambda x: x[0].replace('wav', 'embed').replace(x[1] + '/', ''), axis=1)

/search/hadoop02/suanfa/anaconda3/envs/tacotron2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [223]:
magic_list.speaker_emb_path[0], magic_list.wav_path[0]

('/search/hadoop07/wuzijun/voice_data/Magicdata/embed/37_5622_20170913203118.embed',
 '/search/hadoop07/wuzijun/voice_data/Magicdata/wav/37_5622/37_5622_20170913203118.wav')

In [224]:
magic_trans = magic_list[['wav_path', 'pinyin', 'SpeakerID', 'speaker_emb_path']].copy()

In [225]:
magic_train, magic_val = train_test_split(magic_trans, test_size=300)

In [226]:
len(magic_train), len(magic_val)

(609252, 300)

In [41]:
magic_train = pd.read_csv(path_magic + '/audio_text_train_filelist.txt', header=None, sep='|', 
                          names=['wav_path', 'pinyin', 'Speaker_name', 'speaker_emb_path'])

In [42]:
magic_val = pd.read_csv(path_magic + '/audio_text_val_filelist.txt', header=None, sep='|', 
                          names=['wav_path', 'pinyin', 'Speaker_name', 'speaker_emb_path'])

In [43]:
magic_train.speaker_emb_path[0]

'/search/hadoop07/wuzijun/voice_data/Magicdata/embed14_4028_20170819170219'

In [44]:
magic_train.speaker_emb_path = magic_train.speaker_emb_path.apply(lambda x: x.replace('Magicdata/embed', 'Magicdata/embed/'))

In [45]:
magic_val.speaker_emb_path = magic_val.speaker_emb_path.apply(lambda x: x.replace('Magicdata/embed', 'Magicdata/embed/'))

In [46]:
magic_train.to_csv(path_magic + '/audio_text_train_filelist.txt', header=None, sep='|', index=False)

In [47]:
magic_val.to_csv(path_magic + '/audio_text_val_filelist.txt', header=None, sep='|', index=False)

### Biaobei

In [53]:
biaobei_train = pd.read_csv('/search/hadoop07/wuzijun/voice_data/Biaobei/audio_text_train_filelist.txt', 
                            sep='|', header=None, names=['wav_path', 'pinyin', 'speaker_name', 'speaker_emb_path'])

In [54]:
biaobei_val = pd.read_csv('/search/hadoop07/wuzijun/voice_data/Biaobei/audio_text_val_filelist.txt', 
                            sep='|', header=None, names=['wav_path', 'pinyin', 'speaker_name', 'speaker_emb_path'])

In [62]:
biaobei_train.speaker_emb_path = biaobei_train.speaker_emb_path.apply(lambda x: x.replace('Biaobei/', 'Biaobei/embed/'))

In [63]:
biaobei_val.speaker_emb_path = biaobei_val.speaker_emb_path.apply(lambda x: x.replace('Biaobei/', 'Biaobei/embed/'))

In [64]:
biaobei_val.speaker_emb_path[0]

'/search/hadoop07/wuzijun/voice_data/Biaobei/embed/002937'

In [255]:
biaobei_train['speaker_name'] = 'Biaobei'
biaobei_val['speaker_name'] = 'Biaobei'

In [256]:
biaobei_train['speaker_emb_path'] = biaobei_train.wav_path.apply(lambda x: x.replace('Wave', 'wav').replace('wav', 'embed'))

In [257]:
biaobei_val['speaker_emb_path'] = biaobei_val.wav_path.apply(lambda x: x.replace('Wave', 'wav').replace('wav', 'embed'))

In [258]:
biaobei_train.speaker_emb_path[0], biaobei_train.wav_path[0]

('/search/hadoop07/wuzijun/voice_data/Biaobei/embed/009061.embed',
 '/search/hadoop07/wuzijun/voice_data/Biaobei/embed/009061.embed')

In [262]:
biaobei_train.reset_index(inplace=True)

In [268]:
biaobei_train.wav_path = biaobei_train.level_0
biaobei_train.pinyin = biaobei_train.level_1

In [269]:
biaobei_train = biaobei_train[['wav_path', 'pinyin', 'speaker_emb_path']].copy()

In [272]:
biaobei_train['speaker_name'] = 'Biaobei'

In [281]:
biaobei_train.wav_path = biaobei_train.wav_path.apply(lambda x: x.replace('Wave', 'wav'))

In [65]:
biaobei_train[['wav_path', 'pinyin', 'speaker_name', 'speaker_emb_path']].to_csv('/search/hadoop07/wuzijun/voice_data/Biaobei/audio_text_train_filelist.txt', sep='|', index=False, header=False)

In [276]:
biaobei_val.reset_index(inplace=True)

In [277]:
biaobei_val.wav_path = biaobei_val.level_0
biaobei_val.pinyin = biaobei_val.level_1

In [278]:
biaobei_val = biaobei_val[['wav_path', 'pinyin', 'speaker_emb_path']].copy()

In [279]:
biaobei_val['speaker_name'] = 'Biaobei'

In [283]:
biaobei_val.wav_path = biaobei_val.wav_path.apply(lambda x: x.replace('Wave', 'wav'))

In [66]:
biaobei_val[['wav_path', 'pinyin', 'speaker_name', 'speaker_emb_path']].to_csv('/search/hadoop07/wuzijun/voice_data/Biaobei/audio_text_val_filelist.txt', sep='|', index=False, header=False)

## 测试

In [70]:
test1 = np.load('/search/hadoop07/wuzijun/voice_data/Biaobei/embed/009996.npy')

In [77]:
test2 = np.load('/search/hadoop07/wuzijun/voice_data/Biaobei/embed/009991.npy')

In [1]:
import librosa
import IPython
from IPython.display import Audio

In [131]:
path = '/search/hadoop07/wuzijun/voice_data/Magicdata/wav/15_3621/15_3621_20170810140303.wav'

In [132]:
IPython.display.display(Audio(librosa.load(path, sr=16000)[0], rate=16000))

In [107]:
import torch

In [108]:
from torch import nn

In [ ]:
nn.utils.rnn.pad_packed_sequence

In [109]:
a = torch.rand(5, 9, 4)

In [110]:
b = torch.rand(5, 9, 2)

In [111]:
torch.Tensor.expand

RuntimeError: The size of tensor a (4) must match the size of tensor b (2) at non-singleton dimension 2

In [112]:
torch.zeros(1).repeat(3,4)

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [1]:
from encoder.model import SpeakerEncoder

In [2]:
model = SpeakerEncoder('cuda', 'cuda')

In [3]:
import torch

In [4]:
checkpoint = torch.load('encoder/saved_models/pretrained.pt')

In [1]:
import numpy as np

In [2]:
from synthesizer.utils.audio import AudioProcessor

In [3]:
from synthesizer.utils.generic_utils import load_config

In [4]:
c = load_config('synthesizer/config.json')

In [5]:
ap = AudioProcessor(**c.audio)

 > Setting up Audio Processor...
 | > sample_rate:16000
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:12.5
 | > frame_length_ms:50
 | > ref_level_db:20
 | > num_freq:401
 | > power:1.5
 | > preemphasis:0.98
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > sound_norm:False
 | > n_fft:800
 | > hop_length:200
 | > win_length:800


In [6]:
ap._stft_parameters()

(800, 200, 800)

In [7]:
import numpy as np

In [80]:
import librosa

In [84]:
import soundfile

In [141]:
x, sr = soundfile.read('/search/hadoop07/wuzijun/voice_data/Biaobei/wav/000001.wav')

In [162]:
x, len(x)

(array([-3.05175781e-05,  6.10351562e-05,  1.52587891e-04, ...,
         0.00000000e+00,  0.00000000e+00,  3.05175781e-05]), 127680)

In [157]:
x1, sr1 = librosa.load('/search/hadoop07/wuzijun/voice_data/Biaobei/wav/000001.wav', sr=16000)

In [158]:
x1, sr1

(array([1.11684994e-05, 2.35471700e-04, 4.23695339e-04, ...,
        2.75686380e-05, 1.06456755e-05, 1.14009154e-05], dtype=float32), 16000)

In [159]:
len(x1)

42560

In [155]:
x2 = librosa.resample(x, sr, 16000)

In [156]:
x2

array([1.11684963e-05, 2.35471630e-04, 4.23695326e-04, ...,
       2.75686327e-05, 1.06456725e-05, 1.14009170e-05])

In [160]:
len(x2)

42560

In [161]:
x2 == x1

array([False, False, False, ..., False, False, False])

In [23]:
cd /search/hadoop07/wuzijun/voice_data/AIshell/embed/

/search/hadoop07/wuzijun/voice_data/AIshell/embed


In [24]:
pwd

'/search/hadoop07/wuzijun/voice_data/AIshell/embed'

In [7]:
import glob

In [31]:
test = glob.glob('*')

In [32]:
to_rename = [n for n in test if '\n' in n]

In [33]:
to_rename[0]

IndexError: list index out of range

In [34]:
len(to_rename)

0

In [29]:
import os, sys

In [30]:
for n in to_rename:
    new_n = n.replace('\n', '')
    prefix = new_n.replace('.npy', '').split('/')[-1]
    # print(f'{n} => {new_n}')
    os.system(f"mv {prefix}*.npy {new_n}")